In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("Yogeshvars/BayMaxChat")
model = AutoModelForCausalLM.from_pretrained("Yogeshvars/BayMaxChat")
baymax_model_checkpoint = AutoModelForCausalLM.from_pretrained('./GPT2.1e-05.2.0gpu.2023-05-31164902/')

Some weights of the model checkpoint at /Users/mags/Research/PARTNER/output/GPT2.1e-05.2.0gpu.2023-05-31164902/ were not used when initializing GPT2LMHeadModel: ['position_classifier.dense.bias', 'position_classifier.dense.weight', 'position_classifier.out_proj.bias', 'position_classifier.out_proj.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Just Sample 

In [4]:
chat_history = []
input_text = input("User: ")
new_user_input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors='pt')
bot_input_ids = torch.cat([chat_history, new_user_input_ids], dim=-1) if len(chat_history) > 0 else new_user_input_ids
chat_history_ids = model.generate(
    bot_input_ids, max_length=200,
    pad_token_id=tokenizer.eos_token_id,  
    no_repeat_ngram_size=3,       
    do_sample=True, 
    top_k=100, 
    top_p=0.7,
    temperature=0.8
)
print("Input: {}".format(input_text))
print("BayMax: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Input: i feel so alone, i have no friends
BayMax:  I feel you. I have a friend that is like that and it can be really tough.


### EPIOME Sample

In [5]:
import sys
sys.path.insert(0, '/Users/mags/Research/PARTNER/src/')
from lsp_model_rl.rewards import calc_rewards, calc_coherence_score

In [6]:
original_post = ["I don't feel so great today. I need a friend to talk to."]
original_responses = ["I'm here for you.",]
generated_response = ["I am here for you. Let's talk about what's been on your mind and see how we can make you feel better."]
# stupid responses
generated_response1 = ["Why are you happy today? I don't know what to do"]

# coherence 
coherence_score = calc_coherence_score(original_responses, generated_response[0])

coherence_score4 = calc_coherence_score(original_responses, generated_response1[0])
print(coherence_score)
print(coherence_score4)

0.5360442399978638
0.28044718503952026


In [7]:
def calc_scores(seeker_post, original_post, response_post):
    edit_score = calc_rewards(seeker_post, original_post, response_post, _edit=True)
    bleu_score = calc_rewards(seeker_post, original_post, response_post, _bleu=True)
    distinct_score = calc_rewards(seeker_post, original_post, response_post, _distinct=True)
    perplexity_score = calc_rewards(seeker_post, original_post, response_post, _perplexity=True)
    empathy_score = calc_rewards(seeker_post, original_post, response_post, _empathy=True)
    
    return edit_score, bleu_score, distinct_score, perplexity_score, empathy_score

In [9]:

original_posts = ["I don't feel so great today, I need a friend to talk to." ]
original_responses = ["I'm here for you.",]
generated_response = ["I am here for you. Let's talk about what's been on your mind and see how we can make you feel better."]

x = calc_scores(original_posts, original_responses, generated_response)
coherence_score = calc_coherence_score(original_responses, generated_response[0])
print('edit_score: ', x[0])
print('bleu_score: ', x[1])
print('distinct_score: ', x[2])
print('perplexity_score: ', x[3])
print('empathy_score: ', x[5])
print('coherence_score: ', coherence_score)

edit_score:  0.49999
bleu_score:  0.1395078348258848
distinct_score:  10.954535454545455
perplexity_score:  0.4220868036122937
empathy_score:  0.49999
coherence_score:  0.5360442399978638
